In [0]:
# import json

# dbutils.widgets.text("depth", "2", "Depth")
# depth = int(dbutils.widgets.get("depth"))

# dbutils.widgets.text("dimensions",
#     "[\"geoNetwork.country\",\"device.browser\",\"device.deviceCategory\",\"device.operatingSystem\"]",
#     "Dimensions"
# )
# dimensions = json.loads(dbutils.widgets.get("dimensions"))

# dbutils.widgets.text("kpis", 
#     "[{\"name\": \"Users count\",\"sql\": \"approx_count_distinct(fullVisitorId, 0.03)\"},{\"name\": \"Bounce count\",\"sql\": \"sum(totals.bounces)\"},{\"name\": \"Conversion rate\",\"sql\": \"(sum(totals.transactions)/count(*))*100\"}]",
#     "KPIS"
# )
# kpis = json.loads(dbutils.widgets.get("kpis"))

# dbutils.widgets.text("granularity", "daily", "Granularity")
# granularity = dbutils.widgets.get("granularity")

In [0]:
job_config = {
    "source": {
        "start_date": "25/07/2017",
        "end_date": "30/07/2017"
    },
    "granularity": "daily",
    "kpis": [
        {
            "name": "Users count",
            "sql": "approx_count_distinct(fullVisitorId, 0.03)"
        },
        {
            "name": "Bounce count",
            "sql": "sum(totals.bounces)"
        },
        {
            "name": "Conversion rate",
            "sql": "(sum(totals.transactions)/count(*))*100"
        }
    ],
    "dimensions": [
        "geoNetwork.country",
        "device.browser",
        "device.deviceCategory",
        "device.operatingSystem"
    ],
    "depth": 2
}

In [0]:
import configparser

db_properties={}
config = configparser.ConfigParser()
config.read("db_properties.ini")
db_prop = config['postgresql']
db_properties['user']=db_prop['user']
db_properties['password']=db_prop['password']
db_properties['url']=db_prop['url']
db_properties['dbtable']=db_prop['dbtable']
db_properties['driver']=db_prop['driver']

In [0]:
from datetime import date, timedelta
from datetime import datetime

def daterange(start_date, end_date):
    for n in range(int((end_date - start_date).days)):
        yield start_date + timedelta(n)

def get_dataset_for_date(date):
    table = f'bigquery-public-data.google_analytics_sample.ga_sessions_{date}'
    return spark.read.format("bigquery") \
            .option("table",table) \
            .option("project", 'daniel-343806') \
            .option("parentProject", 'daniel-343806') \
            .load() \
            .limit(1)

def get_dataset_for_range(start_date, end_date):
    df = get_dataset_for_date(start_date.strftime("%Y%m%d"))
    for single_date in daterange(start_date + timedelta(days=1), end_date + timedelta(days=1)):
        df = df.union(get_dataset_for_date(single_date.strftime("%Y%m%d")))
    return df 

In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from itertools import combinations

# Initialize a Spark session
spark = SparkSession.builder.appName("DataAggregation").getOrCreate()

start_date = datetime.strptime(job_config["source"]["start_date"], '%d/%m/%Y')
end_date = datetime.strptime(job_config["source"]["end_date"], '%d/%m/%Y')
df = get_dataset_for_range(start_date, end_date)

# Convert the "timestamp" column to a timestamp type
df = df.withColumn("timestamp", to_timestamp(col("date"), "yyyyMMdd"))

dimensions = [com for sub in range(-1, job_config["depth"]) for com in combinations(job_config["dimensions"], sub + 1)]

for dimension in dimensions:
    for kpi in job_config["kpis"]:
        df_grouped = df.groupBy("timestamp", *dimension) \
                        .agg(
                            coalesce(expr(kpi["sql"]), lit(0)) \
                            .alias(kpi["name"])
                        ) \
                        .withColumn("dimensions", concat(
                            lit("["),
                            concat_ws(", ",
                                *[concat_ws("=", lit(d), col(d.split('.')[-1])) for d in dimension]
                            ),
                            lit("]")
                        )) \
                        .withColumn("aggregation", lit(kpi["name"])) \
                        .withColumn("metric", col(kpi["name"])) \
                        .select("timestamp", "aggregation", "metric", "dimensions")
        df_grouped.write \
            .jdbc(
                url=db_properties['url'], 
                table=db_properties['dbtable'],
                mode='append',
                properties=db_properties
            )


In [0]:
spark.read \
    .jdbc(
        url=db_properties['url'], 
        table=db_properties['dbtable'],
        properties=db_properties
    ) \
    .show()

+-------------------+---------------+------+--------------------+
|          timestamp|    aggregation|metric|          dimensions|
+-------------------+---------------+------+--------------------+
|2017-07-26 00:00:00|    Users count|     1|[geoNetwork.count...|
|2017-07-27 00:00:00|    Users count|     1|[geoNetwork.count...|
|2017-07-28 00:00:00|    Users count|     1|[geoNetwork.count...|
|2017-07-29 00:00:00|    Users count|     1|[geoNetwork.count...|
|2017-07-30 00:00:00|    Users count|     1|[geoNetwork.count...|
|2017-07-26 00:00:00|    Users count|     1|[geoNetwork.count...|
|2017-07-27 00:00:00|    Users count|     1|[geoNetwork.count...|
|2017-07-28 00:00:00|    Users count|     1|[geoNetwork.count...|
|2017-07-29 00:00:00|    Users count|     1|[geoNetwork.count...|
|2017-07-30 00:00:00|    Users count|     1|[geoNetwork.count...|
|2017-07-26 00:00:00|   Bounce count|     0|[geoNetwork.count...|
|2017-07-27 00:00:00|   Bounce count|     0|[geoNetwork.count...|
|2017-07-2